## The purpose of this file
Make summary statistics of flattened temporally aggregated network and save it.<br>The aggregate temporal network is made by projecting a bipartite graph of whole time span on hashtag graph.<br>**Note** that the very last time interval, in which the last interaction occured, is excluded.<br>The resultant network should be the same as the temporally aggregated network (**see section 2**) with all edge weight set to one.<br>The nodes to be removed should be determined in the **section 3**.

In [1]:
import sys
sys.path.append('../')
import networkx as nx
import toolbox as tb
import pandas as pd

In [2]:
tag = "kobe"
hashtag = "神戸" # This variable is necessary for data processing in the module 'toolbox'.
timespan = "21-29"
file = f"../data/datasets/{tag}/{tag}_{timespan}.pkl"
df = tb.get_dataframe(hashtag, file)
df.shape

(25808, 4)

In [3]:
start = "2022-11-23T00:00+09:00"
end = "2022-11-23T23:59+09:00"
timespan = "23-23"
start = pd.to_datetime(start)
end = pd.to_datetime(end)
DF = df[(start <= df.index) & (df.index <= end)]

In [4]:
#DF = df.copy()
#DF.shape

In [5]:
tau, snapshots = tb.get_snapshots_closed_intervals(DF, delta='minutes=30')
#tau, snapshots = tb.get_snapshots_closed_intervals(DF, 'hours=1')

In [6]:
G = tb.get_flattened_temporally_aggregated_network(DF, snapshots)

In [7]:
G.remove_node(hashtag)
print(f"Isolates after removal of the searchtag: \n{list(nx.isolates(G))}")
print("These isolate nodes are to be removed.")
G.remove_nodes_from(list(nx.isolates(G)))
assert len(list(nx.isolates(G))) == 0, "There is at least one isolate node left."
print("============================================")
print("The isolate nodes were successfully deleted.")

Isolates after removal of the searchtag: 
['豆皿小皿食卓とワイン', 'ひまわりの太陽', 'ランチ💕', 'apプロ', '짜파게티', 'theflavordesign', 'イノアカラー', '花耶', '2013/10/13', 'だらだらトーク定例会', 'ラベンダーブラウン', 'サツマスギ', 'balletstudiosora', 'いつでもコーヒー']
These isolate nodes are to be removed.
The isolate nodes were successfully deleted.


In [8]:
N = G.number_of_nodes()
L = G.number_of_edges()
braket_k = (2*G.number_of_edges()) / G.number_of_nodes()
braket_C = nx.average_clustering(G)
density = nx.density(G)

In [9]:
fname = f'../data/temporal_network_summary_statistics/{tag}/ssftn_{timespan}_{tau}.pkl'
statistics = pd.DataFrame({r"$N$": N, r"$L$": L, r"<k>":braket_k, r"<C>":braket_C, r"$Density$": density}, index=[hashtag])
statistics.to_pickle(fname)
print(fname)
statistics

../data/temporal_network_summary_statistics/kobe/ssftn_23-23_43.pkl


,$N$,$L$,<k>,<C>,$Density$
神戸,20122,338318,33.626677,0.889721,0.001671
